In [2]:
import pandas as pd
import numpy as np
import datetime
import os
from os import listdir
from os.path import isfile, join
import shutil
import openpyxl as pxl
import random
# Для парсинга
import re
import time
import random
import calendar
import datetime

from datetime import timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import undetected_chromedriver as uc
from fake_useragent import UserAgent
from tqdm import tqdm
import logging

In [6]:
bin_df = pd.read_excel("/home/user/pythonenvs/test-env/FEDOROV_IVAN/PARSING_BIN/Group_B_bins_kz.xlsx",
                       dtype = {'БИН': 'object'})
bin_df.columns = bin_df.iloc[0, :]
bin_df = pd.DataFrame(bin_df.iloc[1:, :].reset_index(drop=True))
bin_df

/home/user/pythonenvs/test-env/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,БИН,Толық атауы,Полное наименование,Дата регистрации,ОКЭД,Негізгі қызмет түрінің атауы,Наименование основного вида деятельности,Втор.ОКЭД,КРП,ККЖ атауы,...,ЭСЖ атауы,Наименование КСЭ,КФС,МНЖ атауы,Наименование КФС,КАТО,Елді мекеннің атауы,Наименование населенного пункта,"Заңды мекен-жайы, Юридический адрес","Басшының ТАӘ, ФИО руководителя"
0,120340004394,"""KOMIRTEC"" ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІКТЕСТІГІ","ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",2012-03-06 00:00:00,05101,Тас көмірді ашық тәсілмен өндіру,Добыча каменного угля открытым способом,NaN,105,Шағын кәсіпорындар (<= 5),...,Ұлттық жеке қаржылық емес корпорациялар – ПАҰ,Национальные частные нефинансовые корпорации –...,19,Кәсіпорындардың мемлекет пен шет ел қатыспағ...,Собственность предприятий без государственного...,751110000,"АЛМАТЫ ҚАЛАСЫ, АЛМАЛЫ АУДАНЫ","Г.АЛМАТЫ, АЛМАЛИНСКИЙ РАЙОН","УЛИЦА ТОЛЕ БИ, 63",ХАСАНОВ РУСЛАН ДАМИРОВИЧ
1,181040010708,"""МАРП-ТРЕЙД ХОЛДИНГ"" ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ С...","ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",2018-10-09 00:00:00,05101,Тас көмірді ашық тәсілмен өндіру,Добыча каменного угля открытым способом,NaN,105,Шағын кәсіпорындар (<= 5),...,Ұлттық жеке қаржылық емес корпорациялар – ПАҰ,Национальные частные нефинансовые корпорации –...,19,Кәсіпорындардың мемлекет пен шет ел қатыспағ...,Собственность предприятий без государственного...,351013100,"ҚАРАҒАНДЫ ОБЛЫСЫ, ҚАРАҒАНДЫ Қ.Ә., ҚАРАҒАНДЫ Қ....","КАРАГАНДИНСКАЯ ОБЛАСТЬ, КАРАГАНДА Г.А., Г.КАРА...","улица Нарманбет Төлепов, строение 8А",РАХИМБЕКОВ КАНАТ КАНЫШЕВИЧ
2,240740008217,"""MEY ENERGY"" ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІКТЕСТІГІ","ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",2024-07-08 00:00:00,05101,Тас көмірді ашық тәсілмен өндіру,Добыча каменного угля открытым способом,NaN,105,Шағын кәсіпорындар (<= 5),...,Ұлттық жеке қаржылық емес корпорациялар – ПАҰ,Национальные частные нефинансовые корпорации –...,19,Кәсіпорындардың мемлекет пен шет ел қатыспағ...,Собственность предприятий без государственного...,711110000,"АСТАНА ҚАЛАСЫ, АЛМАТЫ АУДАНЫ","Г.АСТАНА, РАЙОН АЛМАТЫ","Проспект Тәуелсіздік, здание 32",ӘЛІМ БАҚЫТНҰР
3,180540024981,"""УГОЛЬ ПРОМ ИНВЕСТ"" ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕ...","ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",2018-05-22 00:00:00,05101,Тас көмірді ашық тәсілмен өндіру,Добыча каменного угля открытым способом,NaN,105,Шағын кәсіпорындар (<= 5),...,Ұлттық жеке қаржылық емес корпорациялар – ПАҰ,Национальные частные нефинансовые корпорации –...,19,Кәсіпорындардың мемлекет пен шет ел қатыспағ...,Собственность предприятий без государственного...,351013100,"ҚАРАҒАНДЫ ОБЛЫСЫ, ҚАРАҒАНДЫ Қ.Ә., ҚАРАҒАНДЫ Қ....","КАРАГАНДИНСКАЯ ОБЛАСТЬ, КАРАГАНДА Г.А., Г.КАРА...","ПРОСПЕКТ БУХАР ЖЫРАУ, 32/3",АБДИКАЛИКОВ АЛМАЗ БОЛАТОВИЧ
4,180540006993,"""АҚТАС КӨМІР"" ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІКТЕС...","ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",2018-05-05 00:00:00,05101,Тас көмірді ашық тәсілмен өндіру,Добыча каменного угля открытым способом,NaN,115,Шағын кәсіпорындар (11-15),...,Ұлттық жеке қаржылық емес корпорациялар – ПАҰ,Национальные частные нефинансовые корпорации –...,19,Кәсіпорындардың мемлекет пен шет ел қатыспағ...,Собственность предприятий без государственного...,351013100,"ҚАРАҒАНДЫ ОБЛЫСЫ, ҚАРАҒАНДЫ Қ.Ә., ҚАРАҒАНДЫ Қ....","КАРАГАНДИНСКАЯ ОБЛАСТЬ, КАРАГАНДА Г.А., Г.КАРА...","Проспект Бухар Жырау, строение 57/1",БАБАС НУРЫМ БОЛАТУЛЫ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4777,230640002986,"""FE-26"" ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІКТЕСТІГІ","ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",2023-06-02 00:00:00,07103,Темір кендерін байыту және агломерациялау,Обогащение и агломерация железных руд,NaN,105,Шағын кәсіпорындар (<= 5),...,Ұлттық жеке қаржылық емес корпорациялар – ПАҰ,Национальные частные нефинансовые корпорации –...,19,Кәсіпорындардың мемлекет пен шет ел қатыспағ...,Собственность предприятий без государственного...,711410000,"АСТАНА ҚАЛАСЫ, БАЙҚОҢЫР АУДАНЫ","Г.АСТАНА, РАЙОН БАЙҚОҢЫР","Прос

In [ ]:
import logging
import time
import pandas as pd
import random
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent
import traceback
from IPython.display import display

# === Функции логирования ===
def log_info(msg):
    print(f"[INFO] {msg}")

def log_error(msg):
    print(f"[ERROR] {msg}")

def log_warning(msg):
    print(f"[WARNING] {msg}")

# === Настройки парсинга ===
bin_list = bin_df['БИН']
url = 'https://ba.prg.kz/392410000-rudniy/070440005295-too-rudnenskiy-produkt/'  # Указываем страницу рандомного ИНН
checkpoint_path = "parsed_contacts_F_group_kz.csv"
restart_browser_after = 100  # Перезапуск браузера каждые 100 запросов
max_attempts = 3  # Количество повторных попыток при отсутствии данных
wait_long = 30  # Долгое ожидание, если сайт не грузится

# === Загрузка уже обработанных данных ===
try:
    existing_data = pd.read_csv(checkpoint_path, dtype=str)
    processed_bins = set(existing_data['Bin'].astype(str))
    log_info(f"✅ Загружено {len(processed_bins)} уже обработанных ИНН.")
except FileNotFoundError:
    log_warning("⚠️ Файл с результатами не найден, начинаем с нуля.")
    existing_data = pd.DataFrame(columns=['Bin', 'Phone', 'Email'])
    processed_bins = set()

# === Функция создания настроек Chrome ===
def create_options():
    options = Options()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--headless")  # Безголовый режим
    options.add_argument(f'--user-agent={UserAgent().random}')
    return options

# === Функция перезапуска браузера ===
def restart_driver():
    global driver
    try:
        if 'driver' in globals():
            driver.quit()
        log_info("🔄 Перезапуск WebDriver...")
        service = Service(ChromeDriverManager(driver_version="124.0.6367.201").install())
        options = create_options()
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)
        wait_for_page_load()
        time.sleep(5)
    except Exception as e:
        log_error(f"Ошибка при запуске WebDriver: {e}")
        raise

# === Ожидание полной загрузки страницы ===
def wait_for_page_load(timeout=20):
    """Ждём, пока страница полностью загрузится."""
    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )

# === Проверка, что страница не пустая ===
def check_page_loaded():
    """Если HTML пустой, ждём дольше и пробуем снова."""
    html = driver.page_source.strip()
    if len(html) < 100:
        log_warning("⚠️ Похоже, сайт не загрузился! Ждём 30 секунд...")
        time.sleep(wait_long)
        driver.refresh()
        wait_for_page_load()
        return False
    return True

# === Ожидание элемента ===
def wait_for_element(selector, by=By.XPATH, timeout=15):
    """Ищем элемент. Если не найден, проверяем HTML страницы."""
    try:
        return WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((by, selector))
        )
    except:
        log_error(f"❌ Элемент {selector} не найден. Проверяем HTML...")
        if not check_page_loaded():
            return None
        return None  # Возвращаем None, чтобы потом повторить попытку

def safe_input_text(driver, element, text):
    """Безопасно вводит ИНН, даже если поле заблокировано или скрыто."""
    driver.execute_script("arguments[0].scrollIntoView(true);", element)
    time.sleep(1)

    # Ожидаем, пока поле станет видимым и активным
    for _ in range(3):
        if element.is_displayed() and element.is_enabled():
            break
        # log_warning("⚠️ Поле ввода неактивно или скрыто, ждём...")
        time.sleep(4)

    # Если всё ещё нельзя взаимодействовать → обновляем страницу
    if not (element.is_displayed() and element.is_enabled()):
        # log_warning("🔄 Поле ввода всё ещё неактивно, обновляем страницу...")
        driver.refresh()
        time.sleep(5)
        return

    # Очистка через Selenium
    try:
        element.clear()
        time.sleep(1)
    except:
        # log_warning("⚠️ Не удалось очистить поле через .clear(), пробуем JS...")
        driver.execute_script("arguments[0].value = '';", element)
        time.sleep(1)

    # Ввод ИНН (если можно)
    try:
        element.send_keys(text)
        element.send_keys(Keys.RETURN)
        time.sleep(random.uniform(5, 7))
    except:
        # log_warning("⚠️ .send_keys() не сработал, пробуем JS...")
        driver.execute_script(f"arguments[0].value = '{text}';", element)
        time.sleep(1.5)
        driver.execute_script("arguments[0].dispatchEvent(new Event('input', { bubbles: true }));", element)
        time.sleep(1.5)
        element.send_keys(Keys.RETURN)  # Всё равно пробуем ENTER

def extract_contacts(prev_phone=None, prev_email=None):
    """Собираем телефоны и email, проверяя, что они действительно обновились."""
    attempt = 0
    while attempt < 3:
        time.sleep(7)  # Даем странице прогрузиться

        contact_blocks = driver.find_elements(By.XPATH, "//div[contains(@class, 'contact')]")
        phones, emails = set(), set()

        for block in contact_blocks:
            label = block.find_element(By.CLASS_NAME, "label").text
            if "Телефон" in label:
                phone_links = block.find_elements(By.XPATH, ".//a[contains(@href, 'tel:')]")
                phones.update([phone.text for phone in phone_links])
            elif "E-mail" in label:
                email_links = block.find_elements(By.XPATH, ".//a[contains(@href, 'mailto:')]")
                emails.update([email.text for email in email_links])

        phone = ", ".join(sorted(phones)) if phones else "Не найдено"
        email = ", ".join(sorted(emails)) if emails else "Не найдено"

        # Проверяем, обновились ли данные
        if (phone, email) != (prev_phone, prev_email):
            return phone, email

        log_warning("⚠️ Контакты не изменились, пробуем ещё раз...")
        time.sleep(4)
        attempt += 1

    log_warning("❌ Контакты не изменились даже после 3 попыток, обновляем страницу...")
    driver.refresh()
    wait_for_page_load()
    return extract_contacts()  # Пробуем снова после перезагрузки

# === Сохранение данных (ДОБАВЛЕНИЕ, НЕ ПЕРЕЗАПИСЬ) ===
def save_checkpoint(results):
    """Добавляет новые строки в csv, не перезаписывая старые."""
    if not results:
        return

    df_new = pd.DataFrame(results)
    df_new = df_new[(df_new["Phone"] != "Не найдено") & (df_new["Email"] != "Не найдено")]

    if not df_new.empty:
        df_new.to_csv(checkpoint_path, mode='a', header=not pd.io.common.file_exists(checkpoint_path), index=False, encoding='utf-8-sig')
        log_info(f"💾 Добавлено {len(df_new)} новых записей")
        display(df_new.tail())

# === Основной код ===
restart_driver()
results = []
checkpoint_interval = 100  # Сохранение каждые 100 записей с контактами

for index, inn in enumerate(tqdm(bin_list, desc='Обрабатываем ИНН:')):
    if index > 0 and index % restart_browser_after == 0:
        log_info(f"🔄 Достигнуто {restart_browser_after} ИНН, перезапускаем браузер...")
        restart_driver()

    # 📌 Проверяем, есть ли ИНН в уже обработанных
    if inn in processed_bins:
        row = existing_data[existing_data['Bin'] == inn]
        phone, email = row['Phone'].values[0], row['Email'].values[0]

        # 📌 Если контакты = l.troinova@gmail.com и +77752925649 → парсим заново
        if phone == "+77752925649" and email == "l.troinova@gmail.com":
            log_warning(f"🔄 Повторный парсинг для {inn} (невалидные контакты).")
        else:
            log_info(f"✅ Пропускаем {inn}, уже обработан.")
            continue  # Пропускаем

    input_xpath = "//input[@placeholder='БИН, ИИН, ФИО, Наименование организации']"
    input_field = wait_for_element(input_xpath)
    safe_input_text(driver, input_field, inn)
    
    phone, email = extract_contacts()

    if phone != "Не найдено" or email != "Не найдено":
        results.append({"Bin": inn, "Phone": phone, "Email": email})

    if len(results) >= checkpoint_interval:
        save_checkpoint(results)
        results = []  # Очищаем буфер

driver.quit()
save_checkpoint(results)
log_info("✅ Обработка завершена.")

[WARNING] ⚠️ Файл с результатами не найден, начинаем с нуля.
[INFO] 🔄 Перезапуск WebDriver...


Обрабатываем ИНН::   0%|          | 0/4782 [00:00<?, ?it/s]

[INFO] 🔄 Достигнуто 100 ИНН, перезапускаем браузер...
[INFO] 🔄 Перезапуск WebDriver...
[INFO] 💾 Добавлено 23 новых записей


,Bin,Phone,Email
76,150940003056,"8 7187 33 65 78, 87187337681",gammasarykoltoo@gmail.com
83,050440002910,87782757775,kambar108@gmail.com
88,980140000025,+77078160030,onayev@gmail.com
90,060440002942,"+77292510169, 510100, 510140",zhainagul.kazmoldayeva@kkm.kz
91,981040001726,87132570833,dm01@inbox.ru


[INFO] 🔄 Достигнуто 100 ИНН, перезапускаем браузер...
[INFO] 🔄 Перезапуск WebDriver...
